In [ ]:
import json
import requests
import os

import sagemaker
import boto3

# Connect to AWS Sagemaker Studio
This method works if running notebook inside AWS Sagemaker Studio

In [ ]:
s3client = boto3.client('s3')
sess = sagemaker.Session()
bucket = sess.default_bucket() 
folder_prefix = 'opensource-10k-data'
print(f"Default bucket: {bucket}")

In [ ]:
def upload_data(fileobject, folder_prefix, filename, bucket):
    s3client.upload_file(fileobject, bucket, os.path.join(folder_prefix, filename))
    

# Get data
In order to calculate the 4 Big Growth Rates, we'll pull the necessary data from the company's 10K using [Financial Modeling Prep's API](https://financialmodelingprep.com/developer/docs) and upload it to an AWS S3 bucket called "financial-data"

In [ ]:
company_tickers = ['DPZ','ALK','MAR','SONY','GME','JNJ','BMBL']

In [ ]:
# read in credentials for API
with open('config.json') as json_file:
    creds = json.load(json_file)

In [ ]:
os.makedirs('data/company_profiles') 
os.makedirs('data/income_sheets')
os.makedirs('data/balance_sheets')
os.makedirs('data/cash_flow_statements')

In [ ]:
# for each company ticker in our list, get data and upload to our azure blob storage container 'financial-data'
for company in company_tickers:
    
    # Company Profile
    response = requests.get("https://financialmodelingprep.com/api/v3/profile/{}?apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        local_file_path = f'./data/company_profiles/{company}-profile.json'
        with open(local_file_path, 'w') as outfile:
            json.dump(response.json(), outfile)
        upload_data(local_file_path, 'company_profiles', "company_profile_{}.json".format(company), bucket)
    else:
        print(response)
        
    # Income Sheet Statement
    response = requests.get("https://financialmodelingprep.com/api/v3/income-statement/{}?limit=5&apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        local_file_path = f'./data/income_sheets/{company}-income.json'
        with open(local_file_path, 'w') as outfile:
            json.dump(response.json(), outfile)
        upload_data(local_file_path, 'income_sheets', "income_sheet_{}.json".format(company), bucket)
    else:
        print(response)
        
    # Balance Sheet Statement
    response = requests.get("https://financialmodelingprep.com/api/v3/balance-sheet-statement/{}?limit=5&apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        local_file_path = f'./data/balance_sheets/{company}-balance.json'
        with open(local_file_path, 'w') as outfile:
            json.dump(response.json(), outfile)
        upload_data(local_file_path, 'balance_sheets', "balance_sheet_{}.json".format(company), bucket)
    else:
        print(response)    
    
    # Cash Flow Statement
    response = requests.get("https://financialmodelingprep.com/api/v3/cash-flow-statement/{}?limit=5&apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        local_file_path = f'./data/cash_flow_statements/{company}-cashflow.json'
        with open(local_file_path, 'w') as outfile:
            json.dump(response.json(), outfile)
        upload_data(local_file_path, 'cash_flow_statements', "cash_flow_statement_{}.json".format(company), bucket)      
    else:
        print(response)